# AORC HMS Grid Setup

This notebook demonstrates how to create HMS grid definition files and HRAP cell mapping for gridded precipitation modeling.

**Workflow Overview**:
1. Load outputs from 14a (AORC DSS grids)
2. Create HMS grid definition file (.grid)
3. Create grid cell mapping file (hrapcells)
4. Validate grid setup artifacts

**Prerequisites**:
- Complete **14a_aorc_download.ipynb** first to generate AORC data
- Files required: `aorc_workflow_output/aorc_storm.nc`, `aorc_workflow_output/aorc_storm.dss`

**Series Navigation**:
- **14a**: AORC download and storm catalog
- **14b** (this notebook): HMS grid definition and HRAP mapping
- **14c**: HMS execution and results analysis

In [ ]:
# pip install hms-commander[all]

**For Development**: If working on hms-commander source code, use the `hmscmdr_local` conda environment (editable install) instead of pip install.

## Setup and Imports

In [ ]:
from pathlib import Path
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

# HMS Commander imports
from hms_commander import HmsHuc, HmsGrid

# Use output directory from 14a
output_dir = Path("aorc_workflow_output")

# Verify prerequisite files exist
nc_file = output_dir / "aorc_storm.nc"
dss_file = output_dir / "aorc_storm.dss"

print("Checking prerequisite files from 14a...")
nc_exists = nc_file.exists()
dss_exists = dss_file.exists()

if nc_exists:
    print(f"  [OK] {nc_file.name}: {nc_file.stat().st_size / (1024*1024):.2f} MB")
else:
    print(f"  [MISSING] {nc_file.name}")

if dss_exists:
    print(f"  [OK] {dss_file.name}: {dss_file.stat().st_size / (1024*1024):.2f} MB")
else:
    print(f"  [MISSING] {dss_file.name}")

# Determine if we can proceed with full execution
PREREQUISITES_MET = nc_exists and dss_exists

if not PREREQUISITES_MET:
    print("\n" + "="*60)
    print("NOTE: Prerequisite files from 14a are missing.")
    print("\nTo generate these files, run 14a_aorc_download.ipynb first.")
    print("This requires:")
    print("  - AORC dependencies: pip install hms-commander[aorc]")
    print("  - Internet access to AWS S3 (AORC data)")
    print("\nThis notebook will run in DEMONSTRATION MODE:")
    print("  - Grid definition and HUC operations will execute")
    print("  - Operations requiring AORC data will show expected output")
    print("="*60)
else:
    print("\nAll prerequisites met - running full workflow")

print(f"\nWorkflow started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Load HUC Watersheds

Re-download or load the HUC12 watersheds for the study area. This step works independently of the AORC data.

In [ ]:
# Same study area bounds as 14a
bounds = (-77.71, 41.01, -77.25, 41.22)

print("Loading HUC12 watersheds for study area...")
watersheds = HmsHuc.get_huc12_for_bounds(bounds)

print(f"Loaded {len(watersheds)} HUC12 watersheds")
print(f"Total area: {watersheds['areasqkm'].sum():.1f} km2")

---

## Phase 3a: Create Grid Definition

Create the HMS .grid file that defines the gridded precipitation source.

**Note**: The .grid file is a text file that references the DSS data. It can be created even without the DSS file existing - HMS will validate the reference at runtime.

In [ ]:
# Get grid format information
grid_info = HmsGrid.get_info()
print("HMS Grid File Formats:")
for fmt, desc in grid_info['format'].items():
    print(f"  {fmt}: {desc}")

print("\nhrapcells Format:")
for key, value in grid_info['hrapcells_format'].items():
    print(f"  {key}: {value}")

In [ ]:
# Create output directory if needed
output_dir.mkdir(exist_ok=True)

# Create .grid file
grid_file = output_dir / "aorc.grid"

print("Creating HMS .grid definition...")

grid_result = HmsGrid.create_grid_definition(
    grid_name="AORC_BaldEagle",
    dss_file="aorc_storm.dss",  # Relative path for HMS project
    pathname="/AORC/BALDEAGLE/PRECIP////",
    output_file=grid_file,
    project_name="BaldEagle",
    description="AORC precipitation for Bald Eagle Creek watershed"
)

print(f"Created: {grid_result}")

In [ ]:
# Display .grid file contents
print("Grid Definition File Contents:")
print("=" * 50)
print(grid_result.read_text())

In [ ]:
# Read back and verify
grid_metadata = HmsGrid.get_grid_info(grid_result)

print("Parsed Grid Metadata:")
print(f"  Grid Manager: {grid_metadata['grid_manager']}")
print(f"  Version: {grid_metadata['version']}")
print(f"  Number of Grids: {len(grid_metadata['grids'])}")

if grid_metadata['grids']:
    g = grid_metadata['grids'][0]
    print(f"\n  Grid 1:")
    print(f"    Name: {g['grid_name']}")
    print(f"    Type: {g['grid_type']}")
    print(f"    DSS File: {g['dss_file']}")
    print(f"    Pathname: {g['pathname']}")

---

## Phase 3b: Create Grid Cell Mapping (hrapcells)

Map the AORC grid cells to each HUC12 subbasin, calculating intersection areas and travel lengths.

**Note**: This step requires the AORC NetCDF file to extract grid coordinates.

In [ ]:
# Get grid coordinates from NetCDF (requires prerequisite file)
if PREREQUISITES_MET:
    import xarray as xr

    ds = xr.open_dataset(nc_file)
    lon_dim = 'longitude' if 'longitude' in ds.dims else 'lon' if 'lon' in ds.dims else 'x'
    lat_dim = 'latitude' if 'latitude' in ds.dims else 'lat' if 'lat' in ds.dims else 'y'

    lon_coords = ds[lon_dim].values
    lat_coords = ds[lat_dim].values
    ds.close()

    print(f"Grid dimensions: {len(lon_coords)} x {len(lat_coords)} = {len(lon_coords) * len(lat_coords)} cells")
else:
    print("DEMONSTRATION MODE: AORC NetCDF file not available")
    print("\nExpected output (with real AORC data):")
    print("  Grid dimensions: ~50 x 25 = ~1250 cells")
    print("\nTo generate this, run 14a_aorc_download.ipynb first.")

In [ ]:
# Create geometry dictionary from HUC12 watersheds
# Use first 5 watersheds for demonstration
num_subbasins = min(5, len(watersheds))

subbasin_geometries = {
    row['huc12']: row['geometry']
    for _, row in watersheds.head(num_subbasins).iterrows()
}

print(f"Mapping {num_subbasins} subbasins:")
for huc, geom in subbasin_geometries.items():
    name = watersheds[watersheds['huc12'] == huc]['name'].iloc[0]
    print(f"  {huc}: {name}")

In [ ]:
# Create hrapcells file (requires AORC grid coordinates)
hrapcells_file = output_dir / "hrapcells"

if PREREQUISITES_MET:
    print("Creating grid cell mapping (hrapcells)...")
    print("(Performing spatial intersection of grid cells with subbasins)")

    hrapcells_result = HmsGrid.map_grid_to_subbasins(
        subbasin_geometries=subbasin_geometries,
        grid_coords=(lon_coords, lat_coords),
        output_hrapcells=hrapcells_file
    )

    print(f"\nCreated: {hrapcells_result}")
    print(f"Size: {hrapcells_result.stat().st_size / 1024:.1f} KB")
else:
    print("DEMONSTRATION MODE: Cannot create hrapcells without AORC grid coordinates")
    print("\nExpected workflow (with real AORC data):")
    print("  1. Read grid coordinates from aorc_storm.nc")
    print("  2. Perform spatial intersection: grid cells x subbasins")
    print("  3. Calculate travel lengths to subbasin outlets")
    print("  4. Write hrapcells file with GRIDCELL entries")
    print("\nExpected output format:")
    print("  Parameter Order: xCoord yCoord TravelLength Area")
    print("  SUBBASIN: 020502030101")
    print("  GRIDCELL: 625 315 12.34 0.85")
    print("  GRIDCELL: 626 315 11.87 0.90")
    print("  ...")

In [ ]:
# Read back and summarize
# Initialize variables for use in later cells
cells = {}
total_cells = 0
total_area = 0

if PREREQUISITES_MET and hrapcells_file.exists():
    cells = HmsGrid.read_hrapcells(hrapcells_file)

    print("Grid Cell Mapping Summary:")
    print("=" * 50)
    print(f"{'Subbasin':<20} {'Cells':>8} {'Area (km2)':>12}")
    print("-" * 50)

    for subbasin, cell_list in cells.items():
        area = sum(c['area'] for c in cell_list)
        total_cells += len(cell_list)
        total_area += area
        print(f"{subbasin:<20} {len(cell_list):>8} {area:>12.1f}")

    print("-" * 50)
    print(f"{'TOTAL':<20} {total_cells:>8} {total_area:>12.1f}")
else:
    print("DEMONSTRATION MODE: hrapcells file not created")
    print("\nExpected summary (with real AORC data):")
    print("  Subbasin              Cells    Area (km2)")
    print("  020502030101            87         65.2")
    print("  020502030102           124         93.1")
    print("  ...")

In [ ]:
# Display sample of hrapcells file
if PREREQUISITES_MET and hrapcells_file.exists():
    print("hrapcells File (first 30 lines):")
    print("=" * 50)
    content = hrapcells_file.read_text()
    for i, line in enumerate(content.split('\n')[:30]):
        print(line)
    if len(content.split('\n')) > 30:
        print("...")
else:
    print("DEMONSTRATION MODE: Sample hrapcells format")
    print("=" * 50)
    print("Parameter Order: xCoord yCoord TravelLength Area")
    print("End:")
    print("SUBBASIN:  020502030101")
    print("GRIDCELL:  625  315  12.34  0.85")
    print("GRIDCELL:  626  315  11.87  0.90")
    print("GRIDCELL:  627  315  11.45  0.88")
    print("...")
    print("END:")
    print("SUBBASIN:  020502030102")
    print("GRIDCELL:  630  318  8.21  0.92")
    print("...")

---

## Validation

Verify that all grid setup artifacts have been created correctly.

In [ ]:
# Validate grid files
print("Validation Results:")
print("=" * 50)

# Check grid definition
if grid_file.exists():
    grid_text = grid_file.read_text()
    has_grid_name = "Grid:" in grid_text
    has_dss_ref = "Filename:" in grid_text
    has_pathname = "Pathname:" in grid_text
    
    if has_grid_name and has_dss_ref and has_pathname:
        print("[PASS] Grid definition file is valid")
    else:
        print("[WARN] Grid definition may be incomplete")
else:
    print("[FAIL] Grid definition file not found")

# Check hrapcells
if PREREQUISITES_MET:
    if hrapcells_file.exists():
        cells = HmsGrid.read_hrapcells(hrapcells_file)
        if len(cells) > 0 and all(len(v) > 0 for v in cells.values()):
            print(f"[PASS] hrapcells file has {len(cells)} subbasins with valid cell mappings")
        else:
            print("[WARN] hrapcells file may have empty subbasins")
    else:
        print("[FAIL] hrapcells file not found")
else:
    print("[SKIP] hrapcells validation - prerequisites not met")

# Check DSS file
if dss_file.exists() and dss_file.stat().st_size > 0:
    print(f"[PASS] DSS grid file exists ({dss_file.stat().st_size / (1024*1024):.2f} MB)")
else:
    print("[SKIP] DSS grid file not found - run 14a_aorc_download.ipynb to generate")

---

## Summary

This notebook demonstrated:
1. Creating HMS grid definition files (.grid)
2. Mapping grid cells to subbasins (hrapcells)
3. Validating grid setup artifacts

**Output Files Created**:
- `aorc.grid` - HMS grid definition
- `hrapcells` - Grid cell mapping for subbasins (if prerequisites met)

**Next Steps**:
- Continue to **14c_aorc_hms_execution.ipynb** to run HMS with gridded precipitation

In [ ]:
# Summary of outputs
print("=" * 60)
if PREREQUISITES_MET:
    print("PHASE 3 COMPLETE: HMS GRID SETUP")
else:
    print("PHASE 3 PARTIAL: HMS GRID SETUP (demonstration mode)")
print("=" * 60)

print(f"\nGrid Definition:")
print(f"  File: {grid_file.name}")
if grid_metadata['grids']:
    g = grid_metadata['grids'][0]
    print(f"  Grid Name: {g['grid_name']}")
    print(f"  DSS Pathname: {g['pathname']}")

if PREREQUISITES_MET:
    print(f"\nGrid Cell Mapping:")
    print(f"  File: {hrapcells_file.name}")
    print(f"  Subbasins: {len(cells)}")
    print(f"  Total cells: {total_cells}")
    print(f"  Total area: {total_area:.1f} km2")
else:
    print(f"\nGrid Cell Mapping:")
    print(f"  [NOT CREATED] - Run 14a_aorc_download.ipynb first")

print(f"\nOutput Files:")
if output_dir.exists():
    for f in sorted(output_dir.glob("*")):
        if f.is_file():
            print(f"  {f.name}: {f.stat().st_size / 1024:.1f} KB")
else:
    print("  (output directory not found)")

print(f"\nWorkflow completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

if PREREQUISITES_MET:
    print("\n--> Continue to 14c_aorc_hms_execution.ipynb")
else:
    print("\n--> To run full workflow: First complete 14a_aorc_download.ipynb")